# 🚀 Trading Strategy Research & Development Lab

Bu notebook, LLM destekli strateji geliştirme döngünü uygulamaya dökmek için tasarlandı.

## 📋 LLM'in Rolü

1. **Veri → Analiz**: Fikir üretme, strateji iskeleti oluşturma
2. **Strateji → Kod**: Python kodu yazma, prototip oluşturma
3. **Kod → Test**: Backtest çalıştırma, metrik hesaplama
4. **Test → Optimize**: Parametre optimizasyonu, grid search
5. **Optimize → Otomasyon**: Bot entegrasyonu, canlı test
6. **Gerçek Zamanlı**: Dinamik strateji güncellemeleri

## 🎯 Bu Notebook'ta Yapacaklarımız

- ✅ Esnek parametre girişi (sembol, timeframe, vb.)
- ✅ Çoklu varlık desteği (BTC, NVDA, altın, vb.)
- ✅ Teknik gösterge analizi (EMA, RSI, MACD)
- ✅ Backtest motoru
- ✅ Performans metrikleri (Sharpe, win rate, drawdown)
- ✅ Görselleştirme (equity curve, sinyal grafiği)
- ✅ Parametre optimizasyonu (grid search)
- ✅ Mevcut bot sistemine entegrasyon

---

## 🤖 LLM Prompt Örnekleri

### Strateji Fikri İçin:
```
"BTC/USDT için RSI 14 ve EMA 50 kullanan bir mean reversion stratejisi öner.
Stop-loss %2, take-profit %4 olsun. Python kodunu yaz."
```

### Optimizasyon İçin:
```
"EMA fast parametresini 10-30 arasında, slow parametresini 30-100 arasında 
test eden bir grid search kodu yaz. En yüksek Sharpe ratio'yu bul."
```

### Risk Yönetimi İçin:
```
"Volatilite arttığında risk_per_trade'i azaltan, 
düştüğünde artıran dinamik bir risk yönetimi fonksiyonu yaz."
```

## 📦 1. Setup ve Kütüphaneler

Önce gerekli kütüphaneleri import edelim ve sistemimizi hazırlayalım.

In [ ]:
# Import temel kütüphaneler
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Mevcut bot modüllerini import et
sys.path.insert(0, os.path.abspath('..'))

from bot.market_data import YFinanceMarketDataClient, PaperExchangeClient
from bot.strategy import StrategyConfig, compute_indicators, generate_signal
from bot.backtesting import Backtester, BacktestResult
from bot.exchange import ExchangeClient
import ta

# Plotting için stil ayarları
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Tüm kütüphaneler yüklendi!")
print(f"📅 Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## ⚙️ 2. Strateji Parametreleri

Burada stratejini test etmek için parametreleri ayarlayabilirsin. LLM'e "bu parametreleri optimize et" diyerek farklı kombinasyonlar deneyebilirsin.

In [ ]:
# ============================================================
# BACKTEST MODU - Parametreler
# ============================================================

# Varlık ve Zaman Ayarları
SYMBOL = "BTC/USDT"          # BTC/USDT, NVDA, GC=F (altın), AAPL, vb.
ASSET_TYPE = "crypto"         # crypto, equity, commodity
TIMEFRAME = "1h"             # 1m, 5m, 15m, 1h, 4h, 1d
LOOKBACK = 1000              # Kaç mum geriye gidilsin?

# Başlangıç Sermayesi
STARTING_BALANCE = 10000     # USD

# 📊 Strateji Parametreleri
EMA_FAST = 12                # Hızlı EMA periyodu
EMA_SLOW = 26                # Yavaş EMA periyodu
RSI_PERIOD = 14              # RSI periyodu
RSI_OVERBOUGHT = 70          # RSI aşırı alım seviyesi
RSI_OVERSOLD = 30            # RSI aşırı satım seviyesi

# 💰 Risk Yönetimi
RISK_PER_TRADE_PCT = 1.0     # Her işlemde riskedilen sermaye yüzdesi
STOP_LOSS_PCT = 2.0          # Stop-loss yüzdesi
TAKE_PROFIT_PCT = 4.0        # Take-profit yüzdesi

# Parametreleri göster
params = {
    "Symbol": SYMBOL,
    "Asset Type": ASSET_TYPE,
    "Timeframe": TIMEFRAME,
    "Lookback": LOOKBACK,
    "Starting Balance": f"${STARTING_BALANCE:,.2f}",
    "EMA Fast": EMA_FAST,
    "EMA Slow": EMA_SLOW,
    "RSI Period": RSI_PERIOD,
    "Risk per Trade": f"{RISK_PER_TRADE_PCT}%",
    "Stop Loss": f"{STOP_LOSS_PCT}%",
    "Take Profit": f"{TAKE_PROFIT_PCT}%"
}

print("="*60)
print("📋 BACKTEST PARAMETRELERİ")
print("="*60)
for key, value in params.items():
    print(f"  {key:20s}: {value}")
print("="*60)

## 📡 3. Veri Çekme

Seçilen varlık türüne göre otomatik olarak doğru veri kaynağını kullanıyoruz:
- **Crypto**: Paper Exchange (sentetik veri) veya CCXT (gerçek veri)
- **Equity/Commodity**: yfinance (Yahoo Finance)

In [ ]:
# Veri kaynağını seç
if ASSET_TYPE == "crypto":
    # Crypto için Paper Exchange kullan (sentetik veri)
    client = PaperExchangeClient()
    print("📊 Veri kaynağı: Paper Exchange (sentetik veri)")
else:
    # Equity/Commodity için yfinance kullan
    client = YFinanceMarketDataClient()
    print(f"📊 Veri kaynağı: Yahoo Finance ({ASSET_TYPE})")

# Veriyi çek
print(f"🔄 {SYMBOL} için {LOOKBACK} mum çekiliyor... ({TIMEFRAME})")
ohlcv_df = client.fetch_ohlcv(SYMBOL, TIMEFRAME, LOOKBACK)

# Veriyi kontrol et
print(f"✅ {len(ohlcv_df)} mum çekildi")
print(f"📅 Tarih aralığı: {ohlcv_df.index[0]} → {ohlcv_df.index[-1]}")
print(f"💵 İlk fiyat: ${ohlcv_df['close'].iloc[0]:,.2f}")
print(f"💵 Son fiyat: ${ohlcv_df['close'].iloc[-1]:,.2f}")
print(f"📈 Değişim: {((ohlcv_df['close'].iloc[-1] / ohlcv_df['close'].iloc[0]) - 1) * 100:.2f}%")

# İlk birkaç satırı göster
print("\n📋 Veri Önizlemesi:")
ohlcv_df.head()

## 📈 4. Teknik Göstergeler

EMA ve RSI göstergelerini hesaplayalım. LLM'e "MACD ekle" veya "Bollinger Bands ekle" diyerek genişletebilirsin.

In [ ]:
# StrategyConfig nesnesi oluştur
config = StrategyConfig(
    symbol=SYMBOL,
    timeframe=TIMEFRAME,
    ema_fast=EMA_FAST,
    ema_slow=EMA_SLOW,
    rsi_period=RSI_PERIOD,
    rsi_overbought=RSI_OVERBOUGHT,
    rsi_oversold=RSI_OVERSOLD,
    risk_per_trade_pct=RISK_PER_TRADE_PCT,
    stop_loss_pct=STOP_LOSS_PCT / 100.0,  # Yüzdeyi ondalığa çevir
    take_profit_pct=TAKE_PROFIT_PCT / 100.0
)

# Göstergeleri hesapla
df_with_indicators = compute_indicators(ohlcv_df.copy(), config)

print("✅ Teknik göstergeler hesaplandı!")
print(f"📊 Toplam {len(df_with_indicators.columns)} sütun var")
print(f"📋 Sütunlar: {', '.join(df_with_indicators.columns.tolist())}")

# Son birkaç satırı göster
print("\n📋 Göstergeler Önizlemesi:")
df_with_indicators[['close', 'ema_fast', 'ema_slow', 'rsi']].tail(10)

## 📊 5. Görselleştirme: Fiyat + EMA + RSI

Stratejinin görsel analizi için fiyat, EMA'lar ve RSI'ı çizelim.

In [ ]:
# Grafik oluştur
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Alt grafik 1: Fiyat ve EMA'lar
ax1.plot(df_with_indicators.index, df_with_indicators['close'], label='Close Price', linewidth=1.5, color='black')
ax1.plot(df_with_indicators.index, df_with_indicators['ema_fast'], label=f'EMA {EMA_FAST}', linewidth=1, alpha=0.7, color='blue')
ax1.plot(df_with_indicators.index, df_with_indicators['ema_slow'], label=f'EMA {EMA_SLOW}', linewidth=1, alpha=0.7, color='red')
ax1.set_ylabel('Price ($)', fontsize=12, fontweight='bold')
ax1.set_title(f'{SYMBOL} - Fiyat ve EMA Göstergeleri', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Alt grafik 2: RSI
ax2.plot(df_with_indicators.index, df_with_indicators['rsi'], label=f'RSI {RSI_PERIOD}', linewidth=1.5, color='purple')
ax2.axhline(y=RSI_OVERBOUGHT, color='red', linestyle='--', linewidth=1, alpha=0.7, label=f'Overbought ({RSI_OVERBOUGHT})')
ax2.axhline(y=RSI_OVERSOLD, color='green', linestyle='--', linewidth=1, alpha=0.7, label=f'Oversold ({RSI_OVERSOLD})')
ax2.axhline(y=50, color='gray', linestyle=':', linewidth=1, alpha=0.5)
ax2.fill_between(df_with_indicators.index, RSI_OVERBOUGHT, 100, alpha=0.1, color='red')
ax2.fill_between(df_with_indicators.index, 0, RSI_OVERSOLD, alpha=0.1, color='green')
ax2.set_ylabel('RSI', fontsize=12, fontweight='bold')
ax2.set_xlabel('Tarih', fontsize=12, fontweight='bold')
ax2.set_title('RSI (Relative Strength Index)', fontsize=14, fontweight='bold')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()

print("✅ Grafikler oluşturuldu!")

## 🎯 6. Sinyal Üretimi

Strateji kurallarına göre alım/satım sinyalleri üretelim:
- **LONG**: EMA fast > EMA slow + RSI < oversold
- **SHORT**: EMA fast < EMA slow + RSI > overbought
- **FLAT**: Diğer durumlar

In [ ]:
# Sinyalleri üret
df_with_signals = df_with_indicators.copy()
df_with_signals['signal'] = df_with_signals.apply(
    lambda row: generate_signal(df_with_signals.loc[:row.name], config),
    axis=1
)

# Sinyal dağılımı
signal_counts = df_with_signals['signal'].value_counts()
print("="*60)
print("📊 SİNYAL DAĞILIMI")
print("="*60)
for signal, count in signal_counts.items():
    percentage = (count / len(df_with_signals)) * 100
    print(f"  {signal:10s}: {count:5d} ({percentage:5.2f}%)")
print("="*60)

# Sinyal değişimlerini bul (alım/satım noktaları)
df_with_signals['signal_changed'] = df_with_signals['signal'] != df_with_signals['signal'].shift(1)
signal_changes = df_with_signals[df_with_signals['signal_changed']]

print(f"\n💡 Toplam {len(signal_changes)} sinyal değişimi tespit edildi")
print("\n📋 İlk 10 Sinyal Değişimi:")
signal_changes[['close', 'ema_fast', 'ema_slow', 'rsi', 'signal']].head(10)

## 🚀 7. Backtest Çalıştırma

Şimdi stratejimizi geçmiş verilerle test edelim ve performansını ölçelim.

In [ ]:
# Backtester oluştur
backtester = Backtester(config, starting_balance=STARTING_BALANCE)

# Backtest çalıştır
print("🔄 Backtest başlatılıyor...")
result = backtester.run(df_with_indicators)

# Sonuçları göster
print("\n" + "="*60)
print("📊 BACKTEST SONUÇLARI")
print("="*60)
print(f"  Başlangıç Bakiyesi  : ${result.starting_balance:,.2f}")
print(f"  Bitiş Bakiyesi      : ${result.final_balance:,.2f}")
print(f"  Toplam P&L          : ${result.total_pnl:,.2f} ({result.total_pnl_pct:.2f}%)")
print(f"")
print(f"  Toplam İşlem        : {result.total_trades}")
print(f"  Kazanan / Kaybeden  : {result.winning_trades} / {result.losing_trades}")
print(f"  Win Rate            : {result.win_rate:.2f}%")
print(f"")
print(f"  Ortalama Kazanç     : ${result.avg_win:.2f}")
print(f"  Ortalama Kayıp      : ${result.avg_loss:.2f}")
print(f"  Profit Factor       : {result.profit_factor:.2f}")
print(f"")
print(f"  Max Drawdown        : ${result.max_drawdown:.2f} ({result.max_drawdown_pct:.2f}%)")
print(f"  Sharpe Ratio        : {result.sharpe_ratio:.2f}")
print("="*60)

# Performans değerlendirmesi
print("\n📈 Performans Değerlendirmesi:")
if result.sharpe_ratio > 1.5:
    print("  ✅ Mükemmel - Sharpe > 1.5")
elif result.sharpe_ratio > 1.0:
    print("  ✅ İyi - Sharpe > 1.0")
elif result.sharpe_ratio > 0.5:
    print("  ⚠️  Orta - Sharpe > 0.5")
else:
    print("  ❌ Zayıf - Sharpe < 0.5")

if result.win_rate > 60:
    print("  ✅ Yüksek Win Rate (>60%)")
elif result.win_rate > 50:
    print("  ✅ İyi Win Rate (>50%)")
else:
    print("  ⚠️  Düşük Win Rate (<50%)")

if result.max_drawdown_pct < 10:
    print("  ✅ Düşük Drawdown (<10%)")
elif result.max_drawdown_pct < 20:
    print("  ⚠️  Orta Drawdown (<20%)")
else:
    print("  ❌ Yüksek Drawdown (>20%)")

## 📉 8. Equity Curve (Sermaye Eğrisi)

Zamanla portföy değerinin nasıl değiştiğini görelim.

In [ ]:
# Equity curve'ü çiz
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Alt grafik 1: Equity Curve
equity_df = pd.DataFrame(result.equity_curve)
ax1.plot(equity_df.index, equity_df['equity'], linewidth=2, color='green', label='Portfolio Value')
ax1.axhline(y=STARTING_BALANCE, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='Starting Balance')
ax1.fill_between(equity_df.index, STARTING_BALANCE, equity_df['equity'], 
                  where=equity_df['equity'] >= STARTING_BALANCE, alpha=0.2, color='green', label='Profit')
ax1.fill_between(equity_df.index, STARTING_BALANCE, equity_df['equity'], 
                  where=equity_df['equity'] < STARTING_BALANCE, alpha=0.2, color='red', label='Loss')
ax1.set_ylabel('Portfolio Value ($)', fontsize=12, fontweight='bold')
ax1.set_title(f'Equity Curve - Total Return: {result.total_pnl_pct:.2f}%', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Alt grafik 2: Drawdown
equity_df['peak'] = equity_df['equity'].cummax()
equity_df['drawdown'] = ((equity_df['equity'] - equity_df['peak']) / equity_df['peak']) * 100
ax2.fill_between(equity_df.index, 0, equity_df['drawdown'], alpha=0.3, color='red')
ax2.plot(equity_df.index, equity_df['drawdown'], linewidth=2, color='darkred', label='Drawdown')
ax2.set_ylabel('Drawdown (%)', fontsize=12, fontweight='bold')
ax2.set_xlabel('İşlem Numarası', fontsize=12, fontweight='bold')
ax2.set_title(f'Drawdown - Max: {result.max_drawdown_pct:.2f}%', fontsize=14, fontweight='bold')
ax2.legend(loc='lower left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Equity curve grafiği oluşturuldu!")

## 💼 9. İşlem Detayları

Tüm alım/satım işlemlerini inceleyelim.

In [ ]:
# İşlem listesini DataFrame'e çevir
if result.trades:
    trades_df = pd.DataFrame([
        {
            'Entry Time': t.entry_time,
            'Exit Time': t.exit_time,
            'Side': t.side,
            'Entry Price': f"${t.entry_price:.2f}",
            'Exit Price': f"${t.exit_price:.2f}",
            'Size': f"{t.size:.4f}",
            'PnL': f"${t.pnl:.2f}",
            'PnL %': f"{t.pnl_pct:.2f}%",
            'Exit Reason': t.exit_reason
        }
        for t in result.trades
    ])
    
    print(f"📋 Toplam {len(trades_df)} işlem gerçekleşti\n")
    print("İlk 10 işlem:")
    display(trades_df.head(10))
    
    print("\n\nSon 10 işlem:")
    display(trades_df.tail(10))
    
    # İşlem analizi
    print("\n" + "="*60)
    print("📊 İŞLEM ANALİZİ")
    print("="*60)
    
    # Side dağılımı
    side_counts = pd.Series([t.side for t in result.trades]).value_counts()
    print(f"\n🔄 İşlem Yönü:")
    for side, count in side_counts.items():
        print(f"  {side:10s}: {count}")
    
    # Exit reason dağılımı
    exit_reasons = pd.Series([t.exit_reason for t in result.trades]).value_counts()
    print(f"\n🚪 Çıkış Sebepleri:")
    for reason, count in exit_reasons.items():
        print(f"  {reason:20s}: {count}")
    
else:
    print("⚠️  Hiç işlem gerçekleşmedi!")

## 🔬 10. Parametre Optimizasyonu (Grid Search)

Burada LLM devreye girer! Farklı parametre kombinasyonlarını test ederek en iyi sonucu bulalım.

**LLM Prompt Örneği:**
```
"EMA fast'ı 10-30, EMA slow'u 30-100, RSI period'u 10-20 arasında test et.
Her kombinasyonun Sharpe ratio'sunu hesapla ve en iyisini bul."
```

In [ ]:
# Optimizasyon parametreleri
ema_fast_range = range(10, 25, 3)      # 10, 13, 16, 19, 22
ema_slow_range = range(30, 70, 10)     # 30, 40, 50, 60
rsi_period_range = range(10, 21, 3)    # 10, 13, 16, 19

# Sonuçları sakla
optimization_results = []

print("🔄 Optimizasyon başlatılıyor...")
print(f"📊 Toplam kombinasyon: {len(ema_fast_range) * len(ema_slow_range) * len(rsi_period_range)}")
print("="*60)

# Grid search
total_combinations = len(ema_fast_range) * len(ema_slow_range) * len(rsi_period_range)
current = 0

for ema_fast in ema_fast_range:
    for ema_slow in ema_slow_range:
        for rsi_period in rsi_period_range:
            current += 1
            
            # Config oluştur
            test_config = StrategyConfig(
                symbol=SYMBOL,
                timeframe=TIMEFRAME,
                ema_fast=ema_fast,
                ema_slow=ema_slow,
                rsi_period=rsi_period,
                rsi_overbought=RSI_OVERBOUGHT,
                rsi_oversold=RSI_OVERSOLD,
                risk_per_trade_pct=RISK_PER_TRADE_PCT,
                stop_loss_pct=STOP_LOSS_PCT / 100.0,
                take_profit_pct=TAKE_PROFIT_PCT / 100.0
            )
            
            # Backtest çalıştır
            try:
                test_backtester = Backtester(test_config, starting_balance=STARTING_BALANCE)
                test_result = test_backtester.run(ohlcv_df.copy())
                
                # Sonucu kaydet
                optimization_results.append({
                    'EMA Fast': ema_fast,
                    'EMA Slow': ema_slow,
                    'RSI Period': rsi_period,
                    'Sharpe': test_result.sharpe_ratio,
                    'Total Return %': test_result.total_pnl_pct,
                    'Win Rate %': test_result.win_rate,
                    'Max DD %': test_result.max_drawdown_pct,
                    'Trades': test_result.total_trades,
                    'Profit Factor': test_result.profit_factor
                })
                
                # İlerleme göster (her 10 kombinasyonda bir)
                if current % 10 == 0:
                    print(f"  {current}/{total_combinations} - EMA({ema_fast},{ema_slow}) RSI({rsi_period}) "
                          f"→ Sharpe: {test_result.sharpe_ratio:.2f} Return: {test_result.total_pnl_pct:.2f}%")
            
            except Exception as e:
                # Hata durumunda atla
                continue

print("="*60)
print(f"✅ Optimizasyon tamamlandı! {len(optimization_results)} kombinasyon test edildi.")

# Sonuçları DataFrame'e çevir
opt_df = pd.DataFrame(optimization_results)

# En iyi 10 kombinasyonu göster (Sharpe'a göre)
print("\n🏆 En İyi 10 Kombinasyon (Sharpe Ratio'ya göre):")
display(opt_df.nlargest(10, 'Sharpe'))

## 📊 11. Optimizasyon Sonuçları Görselleştirme

Heatmap ile hangi parametre kombinasyonlarının daha iyi çalıştığını görelim.

In [ ]:
# Heatmap için pivot table oluştur (ortalama RSI period için)
pivot_data = opt_df.groupby(['EMA Fast', 'EMA Slow'])['Sharpe'].mean().unstack()

# Heatmap çiz
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Sharpe Ratio Heatmap
sns.heatmap(pivot_data, annot=True, fmt='.2f', cmap='RdYlGn', center=0, 
            ax=axes[0], cbar_kws={'label': 'Sharpe Ratio'})
axes[0].set_title('Sharpe Ratio Heatmap (EMA Fast vs EMA Slow)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('EMA Slow', fontsize=12, fontweight='bold')
axes[0].set_ylabel('EMA Fast', fontsize=12, fontweight='bold')

# Return % Heatmap
pivot_return = opt_df.groupby(['EMA Fast', 'EMA Slow'])['Total Return %'].mean().unstack()
sns.heatmap(pivot_return, annot=True, fmt='.1f', cmap='RdYlGn', center=0, 
            ax=axes[1], cbar_kws={'label': 'Return %'})
axes[1].set_title('Total Return % Heatmap (EMA Fast vs EMA Slow)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('EMA Slow', fontsize=12, fontweight='bold')
axes[1].set_ylabel('EMA Fast', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# En iyi kombinasyonu bul
best_combo = opt_df.loc[opt_df['Sharpe'].idxmax()]

print("\n" + "="*60)
print("🏆 EN İYİ PARAMETRE KOMBİNASYONU")
print("="*60)
print(f"  EMA Fast        : {best_combo['EMA Fast']:.0f}")
print(f"  EMA Slow        : {best_combo['EMA Slow']:.0f}")
print(f"  RSI Period      : {best_combo['RSI Period']:.0f}")
print(f"")
print(f"  Sharpe Ratio    : {best_combo['Sharpe']:.2f}")
print(f"  Total Return    : {best_combo['Total Return %']:.2f}%")
print(f"  Win Rate        : {best_combo['Win Rate %']:.2f}%")
print(f"  Max Drawdown    : {best_combo['Max DD %']:.2f}%")
print(f"  Total Trades    : {best_combo['Trades']:.0f}")
print(f"  Profit Factor   : {best_combo['Profit Factor']:.2f}")
print("="*60)

## 💾 12. En İyi Parametreleri Kaydet

Optimizasyon sonucunda bulduğumuz en iyi parametreleri bot sistemine entegre edelim.

In [ ]:
import json
from pathlib import Path

# En iyi parametreleri al
best_params = {
    "symbol": SYMBOL,
    "timeframe": TIMEFRAME,
    "ema_fast": int(best_combo['EMA Fast']),
    "ema_slow": int(best_combo['EMA Slow']),
    "rsi_period": int(best_combo['RSI Period']),
    "rsi_overbought": RSI_OVERBOUGHT,
    "rsi_oversold": RSI_OVERSOLD,
    "risk_per_trade_pct": RISK_PER_TRADE_PCT,
    "stop_loss_pct": STOP_LOSS_PCT / 100.0,
    "take_profit_pct": TAKE_PROFIT_PCT / 100.0
}

# Kayıt yolu
save_path = Path("../data/strategy_config_optimized.json")

# JSON olarak kaydet
with open(save_path, 'w') as f:
    json.dump(best_params, f, indent=2)

print(f"✅ En iyi parametreler kaydedildi: {save_path}")
print("\n📋 Kaydedilen Config:")
print(json.dumps(best_params, indent=2))

print("\n" + "="*60)
print("🚀 SONRAKI ADIMLAR")
print("="*60)
print("1. Bu parametreleri bot'a yükle:")
print(f"   cp {save_path} ../data/strategy_config.json")
print("")
print("2. Bot'u yeniden başlat:")
print("   docker compose restart bot")
print("")
print("3. Canlı test için dry-run modunu dene:")
print("   python ../run_live_trading.py")
print("")
print("4. API'dan yeni parametreleri kontrol et:")
print("   curl http://localhost:8000/strategy | jq")
print("="*60)

## 🤖 13. LLM Entegrasyonu: Dinamik Strateji Güncellemeleri

Bu bölümde LLM'i nasıl kullanabileceğine dair örnekler var.

### 📝 LLM Prompt Şablonları

#### 1️⃣ Strateji İyileştirme
```
Prompt: "Mevcut stratejim EMA(12,26) + RSI(14) kullanıyor. 
Son 1000 mum üzerinde Sharpe ratio 0.8 alıyorum. 
Nasıl iyileştirebilirim? Python kodu ile öner."

LLM Response:
- MACD eklenerek trend gücü ölçülebilir
- ATR ile dinamik stop-loss ayarlanabilir
- Volatilite filtresi eklenebilir (düşük volatilitede işlem yapma)
```

#### 2️⃣ Risk Yönetimi
```
Prompt: "Volatilite arttığında risk_per_trade'i otomatik azaltan, 
düştüğünde artıran bir fonksiyon yaz. ATR kullan."

LLM Response:
def dynamic_risk(atr, atr_ma, base_risk=1.0, min_risk=0.3, max_risk=2.0):
    volatility_ratio = atr / atr_ma
    if volatility_ratio > 1.5:
        return max(min_risk, base_risk * 0.5)
    elif volatility_ratio < 0.7:
        return min(max_risk, base_risk * 1.5)
    return base_risk
```

#### 3️⃣ Piyasa Rejimi Tespiti
```
Prompt: "Son 50 mumda volatilite düşükse ve trend yok ise 
'range-bound' rejimi tespit et. Bu durumda mean reversion stratejisi kullan."

LLM Response:
def detect_market_regime(df, lookback=50):
    recent = df.tail(lookback)
    volatility = recent['close'].std() / recent['close'].mean()
    trend_strength = abs(recent['ema_fast'].iloc[-1] - recent['ema_slow'].iloc[-1]) / recent['close'].iloc[-1]
    
    if volatility < 0.02 and trend_strength < 0.01:
        return "range_bound"
    elif trend_strength > 0.03:
        return "trending"
    else:
        return "neutral"
```

#### 4️⃣ Optimizasyon Öneri
```
Prompt: "Grid search yerine Bayesian Optimization kullan. 
Optuna kütüphanesi ile EMA ve RSI parametrelerini optimize et."

LLM Response:
import optuna

def objective(trial):
    ema_fast = trial.suggest_int('ema_fast', 8, 30)
    ema_slow = trial.suggest_int('ema_slow', 30, 100)
    rsi_period = trial.suggest_int('rsi_period', 7, 30)
    
    config = StrategyConfig(...)
    result = run_backtest(config)
    
    return result.sharpe_ratio

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
```

## 🎯 14. Sonuç ve Öneriler

### ✅ Bu Notebook'ta Neler Yaptık?

1. **Veri çektik** - Crypto, hisse, emtia verilerini otomatik olarak aldık
2. **Gösterge hesapladık** - EMA ve RSI ile teknik analiz yaptık
3. **Sinyal ürettik** - Alım/satım kararları oluşturduk
4. **Backtest çalıştırdık** - Geçmiş verilerle test ettik
5. **Performans ölçtük** - Sharpe, win rate, drawdown hesapladık
6. **Optimize ettik** - Grid search ile en iyi parametreleri bulduk
7. **Görselleştirdik** - Equity curve, heatmap, işlem detayları
8. **Kaydettik** - En iyi parametreleri bot'a entegre ettik

### 📊 Performans Değerlendirmesi

**Mevcut Strateji:**
- Sharpe Ratio: {result.sharpe_ratio:.2f}
- Total Return: {result.total_pnl_pct:.2f}%
- Win Rate: {result.win_rate:.2f}%
- Max Drawdown: {result.max_drawdown_pct:.2f}%

**Optimize Edilmiş Strateji:**
- Sharpe Ratio: {best_combo['Sharpe']:.2f}
- Total Return: {best_combo['Total Return %']:.2f}%
- Win Rate: {best_combo['Win Rate %']:.2f}%
- Max Drawdown: {best_combo['Max DD %']:.2f}%

**İyileştirme:**
- Sharpe: {((best_combo['Sharpe'] - result.sharpe_ratio) / result.sharpe_ratio * 100):.1f}%
- Return: {((best_combo['Total Return %'] - result.total_pnl_pct) / result.total_pnl_pct * 100):.1f}%

### 🚀 Sonraki Adımlar

1. **Walk-Forward Optimization**: Veriyi train/test'e böl, overfitting'i önle
2. **Multiple Timeframes**: 1h + 4h + 1d birleştir
3. **Advanced Indicators**: MACD, Bollinger Bands, ATR ekle
4. **Machine Learning**: XGBoost/LightGBM ile tahmin modeli
5. **Regime Detection**: Trend/range/volatile piyasa tespiti
6. **Portfolio Optimization**: Birden fazla varlıkta risk/getiri dengesi
7. **Live Testing**: Testnet'te gerçek zamanlı test
8. **Production**: Otomatik trading bot'a geç

### 🤖 LLM'in Rolü

- **✅ Kullan**: Fikir üretme, kod yazma, optimizasyon, debugging
- **❌ Kullanma**: Gerçek zamanlı alım/satım kararları
- **🎯 İdeal**: Strateji geliştirme asistanı, kod üretici, parametre optimizer

### 📚 Kaynaklar

- **Bot Sistemini Başlat**: `docker compose up -d`
- **API Dokümantasyonu**: http://localhost:8000/docs
- **Dashboard**: http://localhost:8000/dashboard
- **Mimari Dokümantasyon**: ../ARCHITECTURE.md
- **Hızlı Başlangıç**: ../QUICKSTART_EN.md

---

**🎉 Tebrikler!** LLM destekli strateji geliştirme döngüsünü başarıyla uyguladın!

## 🤖 15. LLM İle Gerçek Zamanlı Analiz

Şimdi yerel LLM'imizi (Mistral) kullanarak gerçek zamanlı strateji analizi yapalım!

In [ ]:
# LLM Client'ı import et
sys.path.insert(0, os.path.abspath('../tools'))
from llm_client import LLMClient

# LLM'i başlat
llm = LLMClient(model="mistral")

# Health check
print("🔍 LLM Bağlantı Kontrolü...")
if llm.health_check():
    print("✅ Mistral-7B hazır!")
    print(f"📍 URL: {llm.base_url}")
    print(f"🤖 Model: {llm.model}")
else:
    print("❌ LLM servisi yanıt vermiyor!")
    print("💡 Terminal'de 'ollama serve' komutuyla servisi başlatın")

### 🎯 A) Backtest Sonuçlarını LLM ile Analiz Et

LLM'e backtest sonuçlarını verip strateji iyileştirme önerileri alalım.

In [ ]:
# Backtest sonuçlarını hazırla
performance_data = {
    "sharpe_ratio": result.sharpe_ratio,
    "win_rate": result.win_rate,
    "max_drawdown_pct": result.max_drawdown_pct,
    "total_pnl_pct": result.total_pnl_pct,
    "total_trades": result.total_trades,
    "profit_factor": result.profit_factor
}

# Piyasa koşullarını hesapla
market_conditions = {
    "volatility": f"{df_with_indicators['close'].pct_change().std() * 100:.2f}%",
    "trend": "bullish" if df_with_indicators['ema_fast'].iloc[-1] > df_with_indicators['ema_slow'].iloc[-1] else "bearish",
    "rsi": f"{df_with_indicators['rsi'].iloc[-1]:.2f}"
}

print("🤖 LLM'e Strateji Analizi Sorgusu Gönderiliyor...\n")
print("="*60)

# LLM'den öneri al
suggestion = llm.suggest_strategy(SYMBOL, performance_data, market_conditions)

print(suggestion)
print("="*60)

### 📊 B) Optimizasyon Sonuçlarını LLM ile Değerlendir

Grid search sonuçlarını LLM'e analiz ettirelim.

In [ ]:
# Mevcut parametreler
current_params = {
    "ema_fast": EMA_FAST,
    "ema_slow": EMA_SLOW,
    "rsi_period": RSI_PERIOD
}

# Optimizasyon geçmişi (en iyi 10'u al)
if len(optimization_results) > 0:
    opt_history = optimization_results[:10]
    
    print("🤖 LLM'e Parametre Optimizasyon Analizi Sorgusu Gönderiliyor...\n")
    print("="*60)
    
    optimization_advice = llm.optimize_parameters(SYMBOL, current_params, opt_history)
    
    print(optimization_advice)
    print("="*60)
else:
    print("⚠️  Önce optimizasyon çalıştırmalısınız (Bölüm 10)")

### 💼 C) İşlemleri LLM ile Açıklat

Son birkaç işlemi LLM'e analiz ettirelim - neden açıldı, neden kapandı?

In [ ]:
# Son 3 işlemi analiz et
if result.trades and len(result.trades) > 0:
    print("🔍 LLM ile İşlem Analizi\n")
    print("="*80)
    
    for i, trade in enumerate(result.trades[-3:], 1):  # Son 3 işlem
        print(f"\n📊 İşlem #{len(result.trades) - 3 + i}")
        print("-" * 80)
        
        # İşlem verilerini hazırla
        trade_data = {
            "side": trade.side,
            "entry_price": trade.entry_price,
            "exit_price": trade.exit_price,
            "pnl_pct": trade.pnl_pct,
            "exit_reason": trade.exit_reason
        }
        
        # Giriş zamanındaki piyasa durumu
        entry_idx = df_with_indicators.index.get_loc(trade.entry_time)
        market_context = {
            "ema_fast": df_with_indicators['ema_fast'].iloc[entry_idx],
            "ema_slow": df_with_indicators['ema_slow'].iloc[entry_idx],
            "rsi": df_with_indicators['rsi'].iloc[entry_idx],
            "price": df_with_indicators['close'].iloc[entry_idx]
        }
        
        # LLM'den açıklama al
        explanation = llm.explain_trade(trade_data, market_context)
        
        print(f"\n🤖 LLM Açıklaması:")
        print(explanation)
        print("-" * 80)
    
    print("\n" + "="*80)
else:
    print("⚠️  Hiç işlem yok, önce backtest çalıştırın")

### 💬 D) LLM'e Serbest Soru Sor

Strateji, piyasa veya herhangi bir konuda LLM'e soru sorabileceğin interaktif hücre.

In [ ]:
# 💡 Bu hücreyi istediğin soruyla değiştir!

# Örnek sorular:
# - "RSI 30'un altındayken alım yapmak mantıklı mı?"
# - "Volatilite yüksek olduğunda stop-loss'u nasıl ayarlamalıyım?"
# - "Bu stratejiye MACD eklesem nasıl olur?"
# - "Mean reversion vs trend-following hangisi bu piyasa için daha uygun?"

your_question = """
{SYMBOL} için mevcut stratejiyi nasıl iyileştirebilirim?
Sharpe ratio: {result.sharpe_ratio:.2f}
Win rate: {result.win_rate:.2f}%
Max drawdown: {result.max_drawdown_pct:.2f}%

Somut öneriler ver.
"""

print("🤖 Soru gönderiliyor...\n")
print("="*80)

answer = llm.ask(your_question)

print(answer)
print("\n" + "="*80)